In [33]:
import pandas as pd
import sqlite3
import os

## 1. Extração (Extract)

Carregamento dos arquivos CSV dos anos 2022, 2023 e 2024.

In [34]:
files = [
    'dados/atendimentoturismo2022.csv',
    'dados/atendimentoturismo2023.csv',
    'dados/atendimentoturismo2024.csv'
]

dfs = []
for f in files:
    df = pd.read_csv(f, sep=';', encoding='utf-8')
    dfs.append(df)
    print(f"Arquivo {f} carregado com sucesso. Linhas: {len(df)}")

df_full = pd.concat(dfs, ignore_index=True)
print(f"Total de registros carregados: {len(df_full)}")

Arquivo dados/atendimentoturismo2022.csv carregado com sucesso. Linhas: 37510
Arquivo dados/atendimentoturismo2023.csv carregado com sucesso. Linhas: 22239
Arquivo dados/atendimentoturismo2024.csv carregado com sucesso. Linhas: 596
Total de registros carregados: 60345
Arquivo dados/atendimentoturismo2023.csv carregado com sucesso. Linhas: 22239
Arquivo dados/atendimentoturismo2024.csv carregado com sucesso. Linhas: 596
Total de registros carregados: 60345


In [35]:
df_full.head()

,idatendimento,qtdadoleslentes,qtdadultos,qtdcriancas,qtdidosos,qtdacompanhantes,qtdturistas,ehacompanhante,estadoorigem,faixaetaria,...,tempoestadia,tipoatendimento,localatendimento,nacionalidade,cidade,deslocamento,informacao,motivoviagem,ano,mes
0,151832,NaN,NaN,NaN,NaN,NaN,NaN,Sim,Nao Preencheu ...,Adulto ...,...,Outros ...,Presencial ...,CAT Praça do Arsenal ...,Internacional ...,...,...,...,...,2022.0,1.0
1,151833,NaN,NaN,NaN,NaN,NaN,NaN,Sim,Sao Paulo ...,Adulto ...,...,5 a 6 dias ...,Presencial ...,CAT Praça do Arsenal ...,Nacional ...,...,...,...,...,2022.0,1.0
2,151834,NaN,NaN,NaN,NaN,NaN,NaN,Sim,Santa Catarina ...,Adulto ...,...,1 a 2 dias ...,Presencial ...,CAT Praça do Arsenal ...,Nacional ...,...,...,...,...,2022.0,1.0
3,151835,NaN,NaN,NaN,NaN,NaN,NaN,Sim,Santa Catarina ...,Adulto ...,...,1 a 2 dias ...,Presencial ...,CAT Praça do Arsenal ...,Nacional ...,...,...,...,...,2022.0,1.0
4,151836,NaN,NaN,NaN,NaN,NaN,NaN,Sim,Minas Gerais ...,Adulto ...,...,Outros ...,Presencial ...,CAT Praça do Arsenal ...,Nacional ...,...,...,...,...,2022.0,1.0


### Inspeção Inicial dos Dados
Visualizando o primeiro registro de forma transposta para entender melhor a estrutura e o conteúdo das colunas.

In [36]:
df_full.iloc[0, :]

idatendimento                                                   151832
qtdadoleslentes                                                    NaN
qtdadultos                                                         NaN
qtdcriancas                                                        NaN
qtdidosos                                                          NaN
qtdacompanhantes                                                   NaN
qtdturistas                                                        NaN
ehacompanhante                                                     Sim
estadoorigem         Nao Preencheu                                 ...
faixaetaria          Adulto                                        ...
tipohospedagem       hotel                                         ...
tipotransporte       Aviao                                         ...
municipointeresse    Nao Preencheu                                 ...
observacao                                                         ...
paisor

## 2. Transformação (Transform)

Limpeza de dados e modelagem das dimensões e tabela fato.

### Análise de Valores Nulos
Verificando a proporção de valores ausentes em cada coluna para definir a estratégia de limpeza.

In [37]:
df_full.isna().sum() / len(df_full)

idatendimento        0.000000
qtdadoleslentes      0.992062
qtdadultos           0.992062
qtdcriancas          0.992062
qtdidosos            0.992062
qtdacompanhantes     0.992062
qtdturistas          0.992062
ehacompanhante       0.003762
estadoorigem         0.003762
faixaetaria          0.003762
tipohospedagem       0.003762
tipotransporte       0.003762
municipointeresse    0.003762
observacao           0.011716
paisorigem           0.019637
sexo                 0.019637
tempoestadia         0.019637
tipoatendimento      0.019637
localatendimento     0.019637
nacionalidade        0.019637
cidade               0.019637
deslocamento         0.019637
informacao           0.019637
motivoviagem         0.019637
ano                  0.019637
mes                  0.019637
dtype: float64

Como as colunas de Quantidade são praticamente nulas elas vão ser removidas, juntamente com restante das linhas que possuirem algum valor nulo.

In [38]:
print(len(df_full))

df_full = df_full.drop(
    [
        "qtdadoleslentes",
        "qtdadultos",
        "qtdcriancas",
        "qtdidosos",
        "qtdacompanhantes",
        "qtdturistas"
    ],
    axis=1
)

df_full = df_full.dropna()

print(len(df_full))

60345
59160


### Ajuste de Tipos de Dados
Garantindo que as colunas de ano e mês sejam inteiros para correta junção com a dimensão tempo.

In [39]:
df_full['ano'] = df_full['ano'].astype(int)
df_full['mes'] = df_full['mes'].astype(int)

### Padronização de Texto
Removendo espaços em branco no início e fim das strings (trim) em todas as colunas de texto. Isso evita que 'SP ' e 'SP' sejam tratados como valores diferentes.

In [40]:
df_full = df_full.apply(lambda col: col.str.strip() if col.dtype == "object" else col)

### Criando Dimensão Tempo

In [41]:
dim_tempo = df_full[['ano', 'mes']].drop_duplicates().reset_index(drop=True)
dim_tempo['id_tempo'] = dim_tempo.index + 1
dim_tempo = dim_tempo[['id_tempo', 'ano', 'mes']]
dim_tempo.head()

,id_tempo,ano,mes
0,1,2022,1
1,2,2022,2
2,3,2022,3
3,4,2022,4
4,5,2022,5


### Criando Dimensão Local

In [42]:
dim_local = df_full[['localatendimento', 'municipointeresse']].drop_duplicates().reset_index(drop=True)
dim_local['id_local'] = dim_local.index + 1
dim_local = dim_local[['id_local', 'localatendimento', 'municipointeresse']]
dim_local.head()

,id_local,localatendimento,municipointeresse
0,1,CAT Praça do Arsenal,Nao Preencheu
1,2,CAT Praça de Boa Viagem,Nao Preencheu
2,3,CAT Praça de Boa Viagem,Recife
3,4,CAT Shopping Center Recife,Recife
4,5,CAT Shopping Center Recife,Nao Preencheu


### Criando Dimensão Perfil Turista

In [43]:
cols_perfil = ['nacionalidade', 'paisorigem', 'estadoorigem', 'cidade', 'sexo', 'faixaetaria']
dim_perfil = df_full[cols_perfil].drop_duplicates().reset_index(drop=True)
dim_perfil['id_perfil'] = dim_perfil.index + 1
dim_perfil = dim_perfil[['id_perfil'] + cols_perfil]
dim_perfil.head()

,id_perfil,nacionalidade,paisorigem,estadoorigem,cidade,sexo,faixaetaria
0,1,Internacional,Franca,Nao Preencheu,,Fem,Adulto
1,2,Nacional,Brasil,Sao Paulo,,Masc,Adulto
2,3,Nacional,Brasil,Santa Catarina,,Fem,Adulto
3,4,Nacional,Brasil,Santa Catarina,,Masc,Adulto
4,5,Nacional,Brasil,Minas Gerais,,Fem,Adulto


### Criando Dimensão Detalhes Viagem

In [44]:
cols_detalhes = ['tipohospedagem', 'tipotransporte', 'motivoviagem', 'tempoestadia', 'tipoatendimento']
dim_detalhes = df_full[cols_detalhes].drop_duplicates().reset_index(drop=True)
dim_detalhes['id_detalhes'] = dim_detalhes.index + 1
dim_detalhes = dim_detalhes[['id_detalhes'] + cols_detalhes]
dim_detalhes.head()

,id_detalhes,tipohospedagem,tipotransporte,motivoviagem,tempoestadia,tipoatendimento
0,1,hotel,Aviao,,Outros,Presencial
1,2,hotel,Aviao,,5 a 6 dias,Presencial
2,3,hotel,Aviao,,1 a 2 dias,Presencial
3,4,outros,Nao Preencheu,,Outros,Presencial
4,5,Nao Preencheu,Onibus,,Nao Informou,Presencial


### Criando Tabela Fato

In [45]:
fato = df_full.merge(dim_tempo, on=['ano', 'mes'], how='left')
fato = fato.merge(dim_local, on=['localatendimento', 'municipointeresse'], how='left')
fato = fato.merge(dim_perfil, on=['nacionalidade', 'paisorigem', 'estadoorigem', 'cidade', 'sexo', 'faixaetaria'], how='left')
fato = fato.merge(dim_detalhes, on=['tipohospedagem', 'tipotransporte', 'motivoviagem', 'tempoestadia', 'tipoatendimento'], how='left')

cols_fato = [
    'idatendimento', 
    'id_tempo', 
    'id_local', 
    'id_perfil', 
    'id_detalhes'
]

fato_atendimentos = fato[cols_fato]
fato_atendimentos.head()

,idatendimento,id_tempo,id_local,id_perfil,id_detalhes
0,151832,1,1,1,1
1,151833,1,1,2,2
2,151834,1,1,3,3
3,151835,1,1,4,3
4,151836,1,1,5,4


## 3. Carga (Load)

Carregando os dados transformados para um banco de dados SQLite.

In [46]:
conn = sqlite3.connect('turismo_dw_etl.db')

dim_tempo.to_sql('dim_tempo', conn, if_exists='replace', index=False)
dim_local.to_sql('dim_local', conn, if_exists='replace', index=False)
dim_perfil.to_sql('dim_perfil_turista', conn, if_exists='replace', index=False)
dim_detalhes.to_sql('dim_detalhes_viagem', conn, if_exists='replace', index=False)
fato_atendimentos.to_sql('fato_atendimentos', conn, if_exists='replace', index=False)

print("Carga concluída com sucesso!")

Carga concluída com sucesso!


## 4. Verificação

Executando uma consulta de exemplo para validar o Data Warehouse.

In [47]:
query = """
SELECT 
    t.ano,
    l.municipointeresse,
    COUNT(*) as total_atendimentos
FROM fato_atendimentos f
JOIN dim_tempo t ON f.id_tempo = t.id_tempo
JOIN dim_local l ON f.id_local = l.id_local
GROUP BY t.ano, l.municipointeresse
ORDER BY t.ano, total_atendimentos DESC
LIMIT 10;
"""

pd.read_sql(query, conn)

,ano,municipointeresse,total_atendimentos
0,2022,Nao Preencheu,20442
1,2022,Recife,11324
2,2022,Olinda,1156
3,2022,Jaboatão dos Guararapes,949
4,2022,Paulista,618
5,2022,Caruaru,348
6,2022,Camaragibe,211
7,2022,Cabo de Santo Agostinho,162
8,2022,Igarassu,133
9,2022,Vitória de Santo Antão,115


In [48]:
conn.close()